# Python for Learning AI Week 2: APIs and Web Communication

Welcome to Week 2 of our Python development series! This week, we'll dive into working with APIs (Application Programming Interfaces) in Python. By the end of this session, you'll understand how to interact with web services, handle API responses, and build robust API integrations.

**Restaurant Metaphor**: Throughout this notebook, we'll compare APIs to a restaurant experience:
- The API is like a restaurant kitchen (the backend)
- You (the client) are a customer placing orders
- Your API requests are like ordering from a menu
- The waiter is the intermediary handling your requests
- API responses are like the food being delivered to your table

This metaphor will help make abstract API concepts more concrete and relatable!

## What You'll Learn
1. Understanding APIs and their role in modern software
2. Making HTTP requests with Python's `requests` library
3. Working with REST APIs and JSON data
4. Handling API authentication and errors
5. Building a simple API client

## Prerequisites
- Basic Python knowledge ([covered in Week 1](../week1/week1_python_basics.ipynb))
- Understanding of dictionaries and JSON
- Familiarity with functions and error handling

Let's start by installing the required packages:

## A Note on Package Management

**Restaurant equivalent**: Before you can place orders at a restaurant, you need to make sure the restaurant has all the ingredients for the menu items (packages) you want to order.

### Why Package Management is Essential

Package management is absolutely critical for successful API development and production code:

1. **Reproducibility**: Ensures your code works the same way across different environments
2. **Dependency Resolution**: Automatically handles complex package relationships
3. **Version Control**: Prevents conflicts between packages with incompatible versions
4. **Collaboration**: Makes it easy for team members to run the same code
5. **Security**: Helps maintain updated packages with security patches

For this course, we use `uv` - a modern, faster Python package installer and resolver.

### Key Benefits of `uv`

- **Speed**: Significantly faster than traditional pip (often 10-100x faster)
- **Reliability**: Improved dependency resolution with fewer conflicts
- **Reproducibility**: Precise version locking for consistent environments
- **Compatibility**: Works with existing requirements files and pyproject.toml

### First: Sync Your Environment

**BEFORE YOU BEGIN**: Always sync your virtual environment with the latest project dependencies:

```bash
# Run this in your terminal before starting work
uv pip sync pyproject.toml
```

This command:
- Updates all packages to match versions in the project configuration (ensures the kitchen has the right ingredients)
- Installs any new dependencies that were added since you last worked (stocks up on new ingredients)
- Removes packages no longer needed by the project (clears out expired ingredients)
- Ensures everyone has the same development environment (all chefs work with the same ingredients)

### Package Management Best Practices

1. **DO NOT** install packages using `!pip install` in notebooks
   - This only installs to the current kernel (like buying ingredients just for one meal)
   - Changes are not persistent across project (ingredients disappear after dinner)
   - Can lead to inconsistencies (different chefs using different ingredients)

2. **DO** install packages using `uv` in your terminal:
   ```bash
   uv pip install package_name
   ```
   - This properly updates your project's inventory (adds to the restaurant's permanent stock)
   - Ensures consistency across the project (all chefs use the same ingredients)

### Need More Help?

- For more details on Python package management, refer to [Week 1 materials](../week1/week1_python_basics.ipynb)
- Ask ChatGPT: "How does uv package manager work in Python?" or "Explain Python virtual environments"
- Check out [uv documentation](https://github.com/astral-sh/uv) for advanced features

In [ ]:
# Import libraries we'll use
import requests
import json

## 1. Understanding APIs

An API (Application Programming Interface) is like a contract between different software components. Think of it as a waiter in a restaurant:
- You (the client) don't need to know how the kitchen (the server) works
- You just need to know how to place an order (make a request)
- The waiter (API) handles communication between you and the kitchen

### Common API Terms:
- **Endpoint**: The URL where an API can be accessed
- **Request**: What you send to the API
- **Response**: What the API sends back
- **HTTP Methods**: GET, POST, PUT, DELETE, etc.
- **Status Codes**: 200 (OK), 404 (Not Found), etc.

Let's start with a simple example using a public API:

In [ ]:
# Let's try a simple API call to JSONPlaceholder (a free testing API)
response = requests.get('https://jsonplaceholder.typicode.com/posts/1')

# Check if the request was successful
if response.status_code == 200:
    # Parse JSON response
    data = response.json()
    print("API Response:")
    print(json.dumps(data, indent=2))
else:
    print(f"Error: Status code {response.status_code}")

# Let's look at the response headers
print("\nResponse Headers:")
for key, value in response.headers.items():
    print(f"{key}: {value}")

## 2. Making Different Types of HTTP Requests

APIs use different HTTP methods for different operations, similar to how a waiter handles different types of requests in a restaurant:

- **GET**: Retrieve data (Like asking the waiter "What's today's special?")
- **POST**: Create new data (Like ordering a new dish: "I'd like the pasta, please.")
- **PUT/PATCH**: Update existing data (Like modifying an order: "Could I get that pasta with chicken instead?") 
- **DELETE**: Remove data (Like canceling an order: "I'd like to cancel the dessert, please.")

Let's set up a base URL for our examples - think of this as choosing which restaurant we'll be visiting:

In [ ]:
base_url = 'https://jsonplaceholder.typicode.com/posts'
print(f"Using API endpoint: {base_url}")

### GET Request: Retrieving Data

The GET method is used to retrieve data from a server. Continuing our restaurant metaphor:

**Restaurant equivalent**: Asking your waiter "Can you tell me about the soup of the day?"

The waiter (API) goes to the kitchen (server), gets information, and returns with details about the soup without changing anything in the kitchen.

Let's create a function to get a post by its ID:

In [ ]:
# GET request (read data)
def get_post(post_id):
    """Retrieve a post by its ID"""
    response = requests.get(f'{base_url}/{post_id}')
    return response.json() if response.ok else None

# Let's try it
post = get_post(1)
print("GET Request Result:")
print(json.dumps(post, indent=2))

### POST Request: Creating Data

The POST method is used to send data to create a new resource. 

**Restaurant equivalent**: Placing a new order with the waiter.

You provide all the details (what dish you want, how you want it cooked, any special requests), and the waiter takes this information to the kitchen where a new dish is created specifically for you. The kitchen (server) responds by creating something new and sending back confirmation (like an order number or estimated time).

Let's create a function to create a new post:

In [ ]:
# POST request (create data)
def create_post(title, body, user_id):
    """Create a new post"""
    new_post = {
        'title': title,
        'body': body,
        'userId': user_id
    }
    response = requests.post(base_url, json=new_post)
    return response.json() if response.ok else None

# Let's try it
new_post = create_post('My New Post', 'This is the content', 1)
print("POST Request Result:")
print(json.dumps(new_post, indent=2))

### PUT Request: Updating Data

The PUT method is used to update an existing resource.

**Restaurant equivalent**: Modifying an order that's already been placed.

Imagine you've ordered a steak (medium rare), but then change your mind. You call the waiter back and say, "I'd like to change my steak to well done and add a side of vegetables." You're updating an existing order, not creating a new one. The waiter takes your complete revised order to the kitchen, replacing the old instructions entirely.

Let's create a function to update a post:

In [ ]:
# PUT request (update data)
def update_post(post_id, title, body, user_id):
    """Update an existing post"""
    updated_post = {
        'id': post_id,
        'title': title,
        'body': body,
        'userId': user_id
    }
    response = requests.put(f'{base_url}/{post_id}', json=updated_post)
    return response.json() if response.ok else None

# Let's try it
updated = update_post(1, 'Updated Title', 'Updated Content', 1)
print("PUT Request Result:")
print(json.dumps(updated, indent=2))

### DELETE Request: Removing Data

The DELETE method is used to remove a resource.

**Restaurant equivalent**: Canceling an order completely.

You realize you need to leave soon, so you tell the waiter, "I need to cancel my dessert order." The waiter goes to the kitchen and removes that order from the system entirely. The kitchen acknowledges the cancellation and stops any preparation that might have started.

Let's create a function to delete a post:

In [ ]:
# DELETE request
def delete_post(post_id):
    """Delete a post by its ID"""
    response = requests.delete(f'{base_url}/{post_id}')
    return response.ok

# Let's try it
success = delete_post(1)
print(f"DELETE Request Result: Success = {success}")

## 3. Handling Authentication

Many APIs require authentication. In our restaurant metaphor, this is like different types of verification the restaurant might require:

**Restaurant equivalents**:
- **API Keys**: Like having a membership card to a exclusive restaurant (The waiter checks your card before taking your order)
- **OAuth tokens**: Similar to making a reservation under someone else's name who has special privileges (The waiter verifies with the manager that you're allowed to use these privileges)
- **Basic authentication**: Like providing your name and phone number to confirm your reservation (Simple identity verification)

**Why Authentication Matters:**
- **Security**: Protects the kitchen's secret recipes (sensitive data)
- **Access Control**: Only paying customers get food (authorized users access resources)
- **Personalization**: The restaurant remembers your preferences (customized user experience)

Let's see how to use different authentication methods:

In [ ]:
# Example with API Key 
def api_key_example():
    """Using an API key in the Authorization header"""
    api_key = "your_api_key_here"  # In real code, get this from environment variables
    
    headers = {
        'Authorization': f'Bearer {api_key}',
        'Content-Type': 'application/json'
    }
    
    # This is just an example URL
    print("Using API key authentication:")
    print(f"  Authorization: Bearer {api_key[:3]}...{api_key[-3:]}" if len(api_key) > 10 else f"  Authorization: Bearer {api_key}")
    
    # In actual use: response = requests.get('https://api.example.com/data', headers=headers)

# Example with Basic Auth
def basic_auth_example():
    """Using username/password authentication"""
    username = "restaurant_user"
    password = "secure_password"  # In real code, get these from environment variables
    
    print("Using Basic authentication:")
    print(f"  With requests: requests.get(url, auth=('{username}', '{password[:2]}****'))")
    
    # In actual use: response = requests.get('https://api.example.com/data', auth=(username, password))

# Run the examples
api_key_example()
print()
basic_auth_example()

## Environment Variables: Best Practices for API Development

When working with APIs, you'll often need to manage sensitive information like:
- API keys
- Authentication tokens
- Credentials
- Database connection strings
- Server configurations

**Restaurant equivalent**: Think of these as your personal information at a restaurant:
- Your membership card or loyalty number (API keys)
- Your reservation confirmation (authentication tokens)
- Your credit card details (credentials)
- Your preferred table location (connection settings)
- Your dietary restrictions (configuration)

You wouldn't want to shout this information across the restaurant (hardcoding in your application), nor would you want it written on a public whiteboard (committed to a public repository).

### Using Environment Variables in Python

Environment variables are key-value pairs that exist outside your application code. They're an ideal solution for managing configuration:

**Restaurant equivalent**: This is like keeping your personal information in your wallet instead of written on your shirt. You can still access it when needed, but it's not visible to everyone.

1. **Benefits:**
   - Separates code from configuration (your meal preference isn't printed on the menu)
   - Improves security by keeping sensitive data out of code (credit card stays in your wallet)
   - Allows different settings across environments (different preferences at different restaurants)

2. **Common Python libraries:**
   - `os` (built-in): Basic environment variable access
   - `dotenv`: Loads variables from .env files
   - `python-decouple`: More advanced configuration management

Let's see how to use them:

In [ ]:
# Basic environment variable handling with os module
import os

# Example: Reading environment variables
def get_api_key():
    """Get API key from environment variable or use a default for development"""
    api_key = os.environ.get('API_KEY')
    if api_key is None:
        print("Warning: API_KEY not found in environment variables!")
        print("Using a placeholder value - this won't work for real APIs")
        return "placeholder_api_key_for_development_only"
    return api_key

# Example of how you would use this in practice
api_key = get_api_key()
print(f"API Key: {api_key}")

# Check for other common environment variables
print("\nCommon environment variables available:")
for env_var in ['USER', 'HOME', 'PATH', 'PYTHON_VERSION']:
    value = os.environ.get(env_var)
    if value:
        # Only show first 30 characters for readability
        display_value = value[:30] + '...' if len(value) > 30 else value
        print(f"- {env_var}: {display_value}")
    else:
        print(f"- {env_var}: Not set")

### Using .env Files

For development, it's common to use `.env` files to store environment variables. The `python-dotenv` package makes this easy.

**Restaurant equivalent**: This is like having your preferences stored on your profile in the restaurant's system. When you arrive, they can look up your preferences without you having to state them each time.

First, you would create a `.env` file in your project root (not committed to version control):

```
# Example .env file
API_KEY=your_secret_key_here
API_URL=https://api.example.com/v1
DEBUG=True
```

In [ ]:
# Import and setup python-dotenv
try:
    from dotenv import load_dotenv
    
    # Load environment variables from .env file
    load_dotenv()  # Take environment variables from .env
    
    print("✓ dotenv package loaded successfully!")
    print("Environment variables from .env file are now available")
    
except ImportError:
    print("✗ The python-dotenv package is not installed.")
    print("\nTo install it, run this in your terminal:")
    print("uv pip install python-dotenv")
    print("\nAfter installing, restart the kernel to use it.")

### Accessing Environment Variables

Now let's create a function to access environment variables with proper defaults and type conversion:

**Restaurant equivalent**: This is like having the waiter check if you have any specific dietary preferences or restrictions in their system before taking your order. If they don't find anything, they'll use standard options.

In [ ]:
try:
    import os
    
    # Example using environment variables for API configuration
    def get_api_config():
        """Get API configuration from environment variables"""
        config = {
            'api_key': os.environ.get('API_KEY', 'default_key_for_dev'),
            'api_url': os.environ.get('API_URL', 'https://api.example.com'),
            'timeout': int(os.environ.get('API_TIMEOUT', '30')),
            'debug': os.environ.get('DEBUG', 'False').lower() in ('true', '1', 't')
        }
        return config
    
except ImportError:
    print("Error importing required modules")

### Displaying Configuration Securely

**Restaurant equivalent**: This is like having the waiter confirm your order without announcing your credit card details to the entire restaurant. They might say "I've got your payment method on file" rather than reading out your card number.

In [ ]:
try:
    # Show the configuration
    api_config = get_api_config()
    print("API Configuration:")
    for key, value in api_config.items():
        # Mask the API key for security
        if key == 'api_key' and value != 'default_key_for_dev':
            display_value = value[:4] + '*' * (len(value) - 4)
        else:
            display_value = value
        print(f"- {key}: {display_value}")
        
except NameError:
    print("Function get_api_config not available. Make sure to run the previous cell.")

### Environment Variables in Production

**Restaurant equivalent**: This is like how different restaurant locations manage their information systems differently, but the end result is the same - your preferences are stored securely.

In production environments, environment variables are typically set differently:

1. **Cloud Platforms:**
   - AWS: Parameter Store, Secrets Manager
   - Azure: App Configuration, Key Vault
   - Google Cloud: Secret Manager
   - Heroku: Config Vars

2. **Container Environments:**
   - Docker: Environment variables in Dockerfile or docker-compose.yml
   - Kubernetes: ConfigMaps and Secrets

3. **CI/CD Systems:**
   - GitHub Actions: Repository secrets
   - Jenkins: Credentials plugin
   - GitLab CI: CI/CD variables

### Best Practices for Environment Variables

**Restaurant equivalent**: These are like the restaurant's policies for handling customer information:

1. **Never hardcode sensitive information** in your code or commit it to repositories (Don't write customer credit card numbers on napkins)
2. **Use different variables for different environments** (Different procedures for take-out vs. dine-in)
3. **Document required environment variables** in your project's README (Train staff on what customer information they need to collect)
4. **Add `.env` files to your `.gitignore`** to prevent accidental commits (Keep customer information in a locked drawer, not on public display)
5. **Use strong encryption** for sensitive values in production (Use a secure safe for storing customer payment details)
6. **Implement validation** to ensure required variables exist before running critical code (Verify you have all needed information before processing an order)

### Conclusion

Environment variables are a powerful tool for API development, allowing you to:
- Keep sensitive credentials secure
- Change configuration without modifying code
- Support different environments (dev, staging, production)
- Follow security best practices

**Restaurant equivalent**: Just as a restaurant needs systems to manage customer preferences, payment details, and location-specific information securely, APIs need environment variables to handle configuration and sensitive data across different contexts.

## 4. Error Handling and Best Practices

When working with APIs, many things can go wrong:
- Network errors
- Authentication failures
- Rate limiting
- Invalid data

**Restaurant equivalent**: Things can go wrong in a restaurant too:
- The kitchen might be closed (server down)
- Your membership card expires (authentication failure)
- The restaurant is too busy to take more orders (rate limiting)
- You ordered something not on the menu (invalid request)

A good waiter knows how to handle these situations gracefully and provide clear explanations to the customer. Similarly, our API client should handle errors gracefully.

Let's build a robust API client with proper error handling:

### Custom Error Class

First, we'll create a custom exception class to handle API-specific errors (like a restaurant's standardized way of communicating problems):

In [ ]:
class APIError(Exception):
    """Custom exception for API errors"""
    def __init__(self, message, status_code=None, response=None):
        self.message = message
        self.status_code = status_code
        self.response = response
        super().__init__(self.message)

### API Client Class

Below, we're creating a robust API client class that will handle all our API interactions:

**Restaurant equivalent**: This is like setting up your initial preferences when you first sit down at a restaurant - specifying any dietary restrictions, preferred server, or seating area before you start ordering.

Our `APIClient` class is designed to:
1. Maintain a persistent connection to the API
2. Handle authentication consistently
3. Process requests and responses in a standardized way
4. Handle errors gracefully with detailed messages
5. Provide convenient methods for different HTTP operations

In [ ]:
class APIClient:
    def __init__(self, base_url, api_key=None):
        self.base_url = base_url
        self.api_key = api_key
        self.session = requests.Session()
        
        # Add authorization header if API key is provided
        if api_key:
            self.session.headers.update({
                'Authorization': f'Bearer {api_key}'
            })
    
    def _make_request(self, method, endpoint, **kwargs):
        """Make an HTTP request with error handling"""
        try:
            # Construct the full URL
            url = f"{self.base_url}/{endpoint}".rstrip('/')
            
            # Make the request using our session
            response = self.session.request(method, url, **kwargs)
            
            # Check for HTTP errors
            response.raise_for_status()
            
            # Return JSON response if available
            return response.json() if response.content else None
            
        except requests.exceptions.HTTPError as e:
            # Handle different HTTP error codes
            if response.status_code == 401:
                raise APIError("Authentication failed", response.status_code, response)
            elif response.status_code == 403:
                raise APIError("Permission denied", response.status_code, response)
            elif response.status_code == 404:
                raise APIError("Resource not found", response.status_code, response)
            elif response.status_code == 429:
                raise APIError("Rate limit exceeded", response.status_code, response)
            else:
                raise APIError(f"HTTP Error: {str(e)}", response.status_code, response)
                
        except requests.exceptions.ConnectionError:
            raise APIError("Failed to connect to API")
        except requests.exceptions.Timeout:
            raise APIError("Request timed out")
        except requests.exceptions.RequestException as e:
            raise APIError(f"API request failed: {str(e)}")
        except ValueError:
            raise APIError("Invalid JSON in response")
            
    # Convenience methods for different HTTP methods
    def get(self, endpoint, params=None):
        return self._make_request('GET', endpoint, params=params)

    def post(self, endpoint, data=None, json=None):
        return self._make_request('POST', endpoint, data=data, json=json)

    def put(self, endpoint, data=None, json=None):
        return self._make_request('PUT', endpoint, data=data, json=json)

    def delete(self, endpoint):
        return self._make_request('DELETE', endpoint)

### Request Handler with Error Management

**Restaurant equivalent**: This is like the waiter's process for handling your orders. The waiter has a standard procedure: take the order to the kitchen, wait for preparation, check if there are problems, and bring back either your food or an explanation of any issues.

Our request handler has been implemented in the `_make_request` method inside our `APIClient` class above. It:
- Makes the actual HTTP request (waiter takes order to kitchen)
- Handles different types of errors (deals with problems professionally)
- Provides detailed error messages (explains why your dish can't be prepared)
- Formats and returns the response (presents your meal properly)

This centralized method ensures all our API requests follow the same error handling patterns.

### Convenience Methods for HTTP Verbs

Our APIClient class includes convenience methods for common HTTP operations:

**Restaurant equivalent**: These are like the standard interactions you have with a waiter:
- `get()`: Asking for information ("What are today's specials?")
- `post()`: Placing new orders ("I'd like to order the pasta")
- `put()`: Changing existing orders ("Please change my side to salad instead of fries")
- `delete()`: Canceling orders ("I'd like to cancel my dessert")

These methods make our code cleaner by abstracting away the details of HTTP requests behind a simple interface.

### Testing Our API Client

**Restaurant equivalent**: Now that we've set up our relationship with the waiter, let's test it out:
1. We'll visit the restaurant (create a client instance)
2. Order a popular dish (make a successful API call)
3. Try to order something that might not be available (try a potentially failing request)
4. See how the waiter handles the situation (test our error handling)

Let's test our robust API client with a real API:

In [ ]:
# Let's use our client with JSONPlaceholder API
try:
    client = APIClient('https://jsonplaceholder.typicode.com')
    
    # Get a post
    post = client.get('posts/1')
    print("Successfully retrieved post:")
    print(json.dumps(post, indent=2))
    
    # Try to get a non-existent post
    post = client.get('posts/999999')
    
except APIError as e:
    print(f"API Error: {e.message}")
    if e.status_code:
        print(f"Status Code: {e.status_code}")

## 5. Register for SPL

**Restaurant equivalent**: Now that you understand how restaurant service works, it's time to make your own reservation (register for the course)!

In [ ]:
# Place holder for Registration for SPL Logic

## 6. Practice Exercise: Build Your Own API Client

**Restaurant equivalent**: Now it's your turn to be the restaurant manager! Design your own service flow for a new restaurant (API client).

Now it's your turn! Create an API client for the [JSONPlaceholder API](https://jsonplaceholder.typicode.com/) that can:

1. Get a list of users (Get a list of regular customers)
2. Get a user's posts (View a customer's order history)
3. Get comments on a post (See feedback on particular dishes)
4. Create a new post (Place a new order)

Requirements:
- Use proper error handling (similar to the Movie API example)
- Include docstrings and comments
- Format the responses nicely
- Add some basic input validation

Just like a good restaurant needs clear procedures for handling orders, customer information, and feedback, your API client should have well-structured methods for each of these operations.

In [ ]:
class JSONPlaceholderClient:
    def __init__(self):
        self.base_url = 'https://jsonplaceholder.typicode.com'
        # Add your code here
    
    def get_users(self):
        """Get all users"""
        # Add your code here
        pass
    
    def get_user_posts(self, user_id):
        """Get posts for a specific user"""
        # Add your code here
        pass
    
    def get_post_comments(self, post_id):
        """Get comments for a specific post"""
        # Add your code here
        pass
    
    def create_post(self, user_id, title, body):
        """Create a new post"""
        # Add your code here
        pass

# Test your implementation here

## Additional Resources

**Restaurant equivalent**: Here's where to find more recipe books and cooking classes to improve your restaurant service!

To learn more about working with APIs in Python:

1. [Requests Library Documentation](https://docs.python-requests.org/)
2. [RESTful API Design Best Practices](https://swagger.io/resources/articles/best-practices-in-api-design/)
3. [Python API Development Fundamentals](https://realpython.com/api-integration-in-python/)
4. [HTTP Status Codes](https://developer.mozilla.org/en-US/docs/Web/HTTP/Status)

## Next Steps

**Restaurant equivalent**: Now that you know the basics of restaurant service:
1. Try working at different types of restaurants (build clients for other APIs)
2. Learn different ways to handle VIP customers (experiment with authentication methods)
3. Design your own signature dishes (create your own API)

Now that you understand the basics of working with APIs in Python:
1. Try building clients for other public APIs
2. Experiment with different authentication methods
3. Consider building your own simple API with Flask or FastAPI